In [ ]:
print 'Report Started:'
!date

# Beamformer observation report

## Files to be processed

In [ ]:
directory = 

## Set up environment

In [ ]:
%pylab inline

In [ ]:
import os
import sys
import numpy as np
from matplotlib import cm
from matplotlib import colors
import psrchive as psr
from coast_guard import cleaners
from coast_guard import clean_utils
from coast_guard import utils

## Load data

In [ ]:
files = []
directory = directory + '/'
for file in os.listdir(directory):
    if file.endswith('.ar'):
        files.append(file)
files.sort()
archives = []
archives = [psr.Archive_load(directory + file) for file in files]
print archives[0]
for i in range(1, len(archives)):
    print archives[i]
    archives[0].append(archives[i])

## Display metadata

In [ ]:
nBin = archives[0].get_nbin()
nChan = archives[0].get_nchan()
nPol = archives[0].get_npol()
nSubint = archives[0].get_nsubint()
obsType = archives[0].get_type()
telescopeName = archives[0].get_telescope()
sourceName = archives[0].get_source()
RA = archives[0].get_coordinates().ra()
Dec = archives[0].get_coordinates().dec()
centreFrequency = archives[0].get_centre_frequency()
bandwidth = archives[0].get_bandwidth()
DM = archives[0].get_dispersion_measure()
RM = archives[0].get_rotation_measure()
isDedispersed = archives[0].get_dedispersed()
isFaradayRotated = archives[0].get_faraday_corrected()
isPolCalib = archives[0].get_poln_calibrated()
dataUnits = archives[0].get_scale()
dataState = archives[0].get_state()
obsDuration = archives[0].integration_length()
receiverName = archives[0].get_receiver_name()
receptorBasis = archives[0].get_basis()
backendName = archives[0].get_backend_name()
lowFreq = archives[0].get_centre_frequency() - archives[0].get_bandwidth() / 2.0
highFreq = archives[0].get_centre_frequency() + archives[0].get_bandwidth() / 2.0
subintDuration = np.rint(archives[0].get_Integration(1).get_duration())

In [ ]:
print 'nbin             Number of pulse phase bins                 %s' % nBin
print 'nchan            Number of frequency channels               %s' % nChan
print 'npol             Number of polarizations                    %s' % nPol
print 'nsubint          Number of sub-integrations                 %s' % nSubint
print 'type             Observation type                           %s' % obsType
print 'site             Telescope name                             %s' % telescopeName
print 'name             Source name                                %s' % sourceName
print 'coord            Source coordinates                         %s%s' % (RA.getHMS(), Dec.getDMS())
print 'freq             Centre frequency (MHz)                     %s' % centreFrequency
print 'bw               Bandwidth (MHz)                            %s' % bandwidth
print 'dm               Dispersion measure (pc/cm^3)               %s' % DM
print 'rm               Rotation measure (rad/m^2)                 %s' % RM
print 'dmc              Dispersion corrected                       %s' % isDedispersed
print 'rmc              Faraday Rotation corrected                 %s' % isFaradayRotated
print 'polc             Polarization calibrated                    %s' % isPolCalib
print 'scale            Data units                                 %s' % dataUnits
print 'state            Data state                                 %s' % dataState
print 'length           Observation duration (s)                   %s' % obsDuration
print 'rcvr:name        Receiver name                              %s' % receiverName
print 'rcvr:basis       Basis of receptors                         %s' % receptorBasis
print 'be:name          Name of the backend instrument             %s' % backendName

## Clean archive, display RFI stats and RFI mask

In [ ]:
cleanRFI = archives[0].clone()
cleaner = cleaners.load_cleaner('surgical')
cleaner.parse_config_string('chan_numpieces=1,subint_numpieces=1,chanthresh=3,subintthresh=3')
cleaner.run(cleanRFI)
weights = cleanRFI.get_weights()
commonCleanRFI = cleanRFI.clone()
commonCleanRFI.dedisperse()
commonCleanRFI.remove_baseline()
commonCleanRFI.centre_max_bin()
i = j = counter = channel = 0
emptyChan = [i for i in range(nChan)]
for i in range(nSubint):
    channel = 0
    del emptyChan[:]
    for j in range(nChan):
        if weights[i][j] == 0.0:
            counter += 1
            channel += 1
            emptyChan.append(j)
    subintProc = (float(channel) / float(nChan)) * 100
    print 'Subint %s has %s channels (%.2f%%) set to zero.' % (i, channel, subintProc)
percentage = (float(counter) / float(weights.size)) * 100
print '%s data points out of %s with weights set to zero.' % (counter, weights.size)
print '%.2f%% data was set to zero.' % (percentage)
fig, ax1 = plt.subplots(1, 1, figsize = [15, 10])
ax1.set_title(sourceName)
ax1.set_title('RFI mask', loc = 'left')
ax1.set_ylabel('Channel number')
ax1.yaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax1.set_xlabel('Subint number')
ax1Secondary = ax1.twinx()
ax1Secondary.set_ylabel('Frequency (MHz)')
ax1Secondary.set_ylim(lowFreq, highFreq)
ax1Secondary.yaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 25))
ax1.imshow(weights.T, origin = 'lower', aspect = 'auto', cmap = colors.ListedColormap(['red', 'white']), interpolation = 'none', extent = (0, nSubint - 1, 0, nChan - 1))

## Plot average profiles (Stokes data)

In [ ]:
cleanProfile = commonCleanRFI.clone()
cleanProfile.tscrunch()
cleanProfile.fscrunch()
cleanProfile.convert_state('Stokes')
cleanProfileData = cleanProfile.get_data()
cleanProfileData = cleanProfileData.squeeze()
cleanMinMaxVals = np.array([cleanProfileData[0, :].max(), cleanProfileData[1, :].max(), cleanProfileData[2, :].max(), cleanProfileData[3, :].max(), cleanProfileData[0, :].min(), cleanProfileData[1, :].min(), cleanProfileData[2, :].min(), cleanProfileData[3, :].min()])
dirtyProfile = archives[0].clone()
dirtyProfile.dedisperse()
dirtyProfile.remove_baseline()
dirtyProfile.centre_max_bin()
dirtyProfile.tscrunch()
dirtyProfile.fscrunch()
dirtyProfile.convert_state('Stokes')
dirtyProfileData = dirtyProfile.get_data()
dirtyProfileData = dirtyProfileData.squeeze()
dirtyMinMaxVals = np.array([dirtyProfileData[0, :].min(), dirtyProfileData[1, :].min(), dirtyProfileData[2, :].min(), dirtyProfileData[3, :].min(), dirtyProfileData[0, :].max(), dirtyProfileData[1, :].max(), dirtyProfileData[2, :].max(), dirtyProfileData[3, :].max()])
fig, ((ax1, ax2)) = plt.subplots(2, 1, sharex = 'col', figsize = [15, 10], tight_layout = 'false')
ax1.set_title(sourceName)
ax1.text(20, dirtyProfileData.max(), 'Unclean data', fontsize = 'large')
ax1.set_ylabel('Flux (a.u.)')
ax1.set_ylim((dirtyMinMaxVals.min() * 1.2), dirtyMinMaxVals.max() + 0.1 * dirtyMinMaxVals.max())
ax1.plot(dirtyProfileData[0, :])
ax1.plot(dirtyProfileData[1, :])
ax1.plot(dirtyProfileData[2, :])
ax1.plot(dirtyProfileData[3, :])
ax1.plot(dirtyProfileData[0, :], 'k')
ax1.plot(dirtyProfileData[1, :], 'r')
ax1.plot(dirtyProfileData[2, :], 'g')
ax1.plot(dirtyProfileData[3, :], 'b')
ax2.text(20, cleanProfileData.max(), 'Clean data', fontsize = 'large')
ax2.set_xlabel('Pulse phase (bins)')
ax2.set_xlim(0, nBin - 1)
ax2.set_ylabel('Flux (a.u.)')
ax2.set_ylim((cleanMinMaxVals.min() * 1.2), cleanMinMaxVals.max() + 0.1 * cleanMinMaxVals.max())
ax2.xaxis.set_ticks(np.arange(0, nBin - 1, 50))
ax2Secondary = ax2.twiny()
ax2Secondary.set_frame_on(True)
ax2Secondary.patch.set_visible(False)
ax2Secondary.xaxis.set_ticks_position('bottom')
ax2Secondary.set_xlabel('Pulse phase (degrees)')
ax2Secondary.xaxis.set_label_position('bottom')
ax2Secondary.spines['bottom'].set_position(('outward', 50))
ax2Secondary.set_xlim(0, 360)
_ = ax2Secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax2.plot(cleanProfileData[0, :], 'k')
ax2.plot(cleanProfileData[1, :], 'r')
ax2.plot(cleanProfileData[2, :], 'g')
ax2.plot(cleanProfileData[3, :], 'b')

## Plot subintegration stacks (Coherency data)

In [ ]:
dirtyStack = archives[0].clone()
dirtyStack.dedisperse()
dirtyStack.remove_baseline()
dirtyStack.centre_max_bin()
dirtyStack.fscrunch()
dirtyStack.pscrunch()
dirtyStack_data = dirtyStack.get_data().squeeze()
cleanStack = commonCleanRFI.clone()
cleanStack.fscrunch()
cleanStack.pscrunch()
cleanStack_data = cleanStack.get_data().squeeze()
fig, ((ax1, ax2)) = plt.subplots(1, 2, sharey = 'row', figsize = [15, 10], tight_layout = 'false')
ax1.set_title(sourceName)
ax1.set_title('Unclean data', loc = 'left')
ax1.set_ylabel('Subint number')
ax1.set_xlabel('Pulse phase (bin)')
ax1.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax1Secondary = ax1.twiny()
ax1Secondary.set_frame_on(True)
ax1Secondary.patch.set_visible(False)
ax1Secondary.xaxis.set_ticks_position('bottom')
ax1Secondary.set_xlabel('Pulse phase (degrees)')
ax1Secondary.xaxis.set_label_position('bottom')
ax1Secondary.spines['bottom'].set_position(('outward', 50))
ax1Secondary.set_xlim(0, 360)
_ = ax1Secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax1.imshow(dirtyStack_data, cmap = cm.afmhot, aspect = 'auto', vmax = dirtyStack_data.max() * 0.75, interpolation = 'none', extent = (0, nBin - 1, 0, nSubint - 1))
ax2.set_title(sourceName)
ax2.set_title('Clean data', loc = 'left')
ax2.set_xlabel('Pulse phase (bin)')
ax2.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax2Secondary = ax2.twiny()
ax2Secondary.set_frame_on(True)
ax2Secondary.patch.set_visible(False)
ax2Secondary.xaxis.set_ticks_position('bottom')
ax2Secondary.set_xlabel('Pulse phase (degrees)')
ax2Secondary.xaxis.set_label_position('bottom')
ax2Secondary.spines['bottom'].set_position(('outward', 50))
ax2Secondary.set_xlim(0, 360)
_ = ax2Secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax2Tertiary = ax2.twinx()
ax2Tertiary.yaxis.set_label_position('right')
ax2Tertiary.set_ylim(0, obsDuration)
ax2Tertiary.set_ylabel('Time (seconds)')
ax2.imshow(cleanStack_data, cmap = cm.afmhot, aspect = 'auto', vmax = cleanStack_data.max() * 0.75, interpolation = 'none', extent = (0, nBin - 1, 0, nSubint - 1))

## Plot phase vs. frequency image of flux (clean Coherency data)

In [ ]:
coherencyFluxPhase = commonCleanRFI.clone()
coherencyFluxPhase.tscrunch()
coherencyFluxPhase_data = coherencyFluxPhase.get_data().squeeze()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = 'col', sharey = 'row', figsize = [15, 10], tight_layout = 'false')
fig.suptitle(sourceName, fontsize = 'large', y = 1.015)
fig.text(0.05, 1.0, 'Clean data', fontsize = 'large')
ax1.set_title('XX')
ax1.set_ylabel('Frequency (MHz)')
ax1.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax1.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax1.imshow(coherencyFluxPhase_data[0, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))
ax2.set_title('YY')
ax2.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax2.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax2Secondary = ax2.twinx()
ax2Secondary.set_ylabel('Channel number')
ax2Secondary.set_ylim(0, nChan - 1)
ax2Secondary.yaxis.set_ticks(np.arange(0, nChan - 1 , 200))
ax2.imshow(coherencyFluxPhase_data[1, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))
ax3.set_title('Re XY')
ax3.set_xlabel('Pulse phase (bin)')
ax3.set_ylabel('Frequency (MHz)')
ax3.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax3.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax3Secondary = ax3.twiny()
ax3Secondary.set_frame_on(True)
ax3Secondary.patch.set_visible(False)
ax3Secondary.xaxis.set_ticks_position('bottom')
ax3Secondary.set_xlabel('Pulse phase (degrees)')
ax3Secondary.xaxis.set_label_position('bottom')
ax3Secondary.spines['bottom'].set_position(('outward', 50))
ax3Secondary.set_xlim(0, 360)
_ = ax3Secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax3.imshow(coherencyFluxPhase_data[2, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))
ax4.set_title('Im XY')
ax4.set_xlabel('Pulse phase (bin)')
ax4.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax4.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax4Secondary = ax4.twinx()
ax4Secondary.set_ylabel('Channel number')
ax4Secondary.set_ylim(0, nChan - 1)
ax4Secondary.yaxis.set_ticks(np.arange(0, nChan - 1 , 200))
ax4Tertiary = ax4.twiny()
ax4Tertiary.set_frame_on(True)
ax4Tertiary.patch.set_visible(False)
ax4Tertiary.xaxis.set_ticks_position('bottom')
ax4Tertiary.set_xlabel('Pulse phase (degrees)')
ax4Tertiary.xaxis.set_label_position('bottom')
ax4Tertiary.spines['bottom'].set_position(('outward', 50))
ax4Tertiary.set_xlim(0, 360)
_ = ax4Tertiary.xaxis.set_ticks(np.arange(0, 360, 20))
ax4.imshow(coherencyFluxPhase_data[3, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))

## Plot phase vs. frequency image of flux (clean Stokes data)

In [ ]:
stokesFluxPhase = commonCleanRFI.clone()
stokesFluxPhase.tscrunch()
stokesFluxPhase.convert_state('Stokes')
stokesFluxPhase_data = stokesFluxPhase.get_data().squeeze()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = 'col', sharey = 'row', figsize = [15, 10], tight_layout = 'false')
fig.suptitle(sourceName, fontsize = 'large', y = 1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize = 'large')
ax1.set_title('I')
ax1.set_ylabel('Frequency (MHz)')
ax1.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax1.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax1.imshow(stokesFluxPhase_data[0, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))
ax2.set_title('Q')
ax2.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax2.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax2Secondary = ax2.twinx()
ax2Secondary.set_ylabel('Channel number')
ax2Secondary.set_ylim(0, nChan - 1)
ax2Secondary.yaxis.set_ticks(np.arange(0, nChan - 1 , 200))
ax2.imshow(stokesFluxPhase_data[1, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))
ax3.set_title('U')
ax3.set_xlabel('Pulse phase (bin)')
ax3.set_ylabel('Frequency (MHz)')
ax3.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax3.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax3Secondary = ax3.twiny()
ax3Secondary.set_frame_on(True)
ax3Secondary.patch.set_visible(False)
ax3Secondary.xaxis.set_ticks_position('bottom')
ax3Secondary.set_xlabel('Pulse phase (degrees)')
ax3Secondary.xaxis.set_label_position('bottom')
ax3Secondary.spines['bottom'].set_position(('outward', 50))
ax3Secondary.set_xlim(0, 360)
_ = ax3Secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax3.imshow(stokesFluxPhase_data[2, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))
ax4.set_title('V')
ax4.set_xlabel('Pulse phase (bin)')
ax4.xaxis.set_ticks(np.arange(0, nBin - 1, 100))
ax4.yaxis.set_ticks(np.arange(0, nChan - 1, 50))
ax4Secondary = ax4.twinx()
ax4Secondary.set_ylabel('Channel number')
ax4Secondary.set_ylim(0, nChan - 1)
ax4Secondary.yaxis.set_ticks(np.arange(0, nChan - 1 , 200))
ax4Tertiary = ax4.twiny()
ax4Tertiary.set_frame_on(True)
ax4Tertiary.patch.set_visible(False)
ax4Tertiary.xaxis.set_ticks_position('bottom')
ax4Tertiary.set_xlabel('Pulse phase (degrees)')
ax4Tertiary.xaxis.set_label_position('bottom')
ax4Tertiary.spines['bottom'].set_position(('outward', 50))
ax4Tertiary.set_xlim(0, 360)
_ = ax4Tertiary.xaxis.set_ticks(np.arange(0, 360, 20))
ax4.imshow(stokesFluxPhase_data[3, :, :], cmap = cm.afmhot, aspect = 'auto', interpolation = 'none', extent = (0, nBin - 1.0, lowFreq, highFreq))

## Plot bandpass (unclean Stokes data)

In [ ]:
dirtyBandpass = archives[0].clone()
dirtyBandpass.tscrunch()
(dirtyBandpassMean, dirtyBandpassVariance) = dirtyBandpass.get_Integration(0).baseline_stats()
dirtyBandpassMaxMinVals = np.array([dirtyBandpassMean[0, :].min(), dirtyBandpassMean[0, :].max(), dirtyBandpassMean[1, :].min(), dirtyBandpassMean[1, :].max()])
dirtyBandpassStokes = archives[0].clone()
dirtyBandpassStokes.convert_state('Stokes')
dirtyBandpassStokes.tscrunch()
(dirtyBandpassStokesMean, dirtyBandpassStokesVariance) = dirtyBandpassStokes.get_Integration(0).baseline_stats()
dirtyBandpassStokesMinMaxVals = np.array([dirtyBandpassStokesMean[0, :].min(), dirtyBandpassStokesMean[0, :].max()])
fig, ((ax1, ax2, ax3 )) = plt.subplots(3, 1, sharex = 'col', figsize = [15, 10], tight_layout = 'false')
ax1.set_title(sourceName)
ax1.set_title('Unclean data', loc = 'left')
ax1.text(np.int(nChan * 0.0125), dirtyBandpassMaxMinVals.max() - (0.25 * dirtyBandpassMaxMinVals.max()), 'XX', fontsize = 'large')
ax1.set_xlim(0, nChan - 1)
ax1.set_ylabel('Flux (a.u.)')
ax1.xaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax1.set_yscale('log')
ax1.plot(dirtyBandpassMean[0, :])
ax2.text(np.int(nChan * 0.0125), dirtyBandpassMaxMinVals.max() - (0.25 * dirtyBandpassMaxMinVals.max()), 'YY', fontsize='large')
ax2.set_xlim(0, nChan - 1)
ax2.set_ylabel('Flux (a.u.)')
ax2.xaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax2.set_yscale('log')
ax2.plot(dirtyBandpassMean[1, :])
ax3.text(np.int(nChan*0.0125), dirtyBandpassMaxMinVals.max() - (0.25 * dirtyBandpassMaxMinVals.max()), 'Stokes I', fontsize='large')
ax3.set_xlabel('Channel number')
ax3.set_ylabel('Flux (a.u.)')
ax3.xaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax3.set_yscale('log')
ax3Secondary = ax3.twiny()
ax3Secondary.set_frame_on(True)
ax3Secondary.patch.set_visible(False)
ax3Secondary.xaxis.set_ticks_position('bottom')
ax3Secondary.set_xlabel('Frequency (MHz)')
ax3Secondary.xaxis.set_label_position('bottom')
ax3Secondary.spines['bottom'].set_position(('outward', 50))
ax3Secondary.set_xlim(lowFreq, highFreq)
_ = ax3Secondary.xaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 50))
ax3.plot(dirtyBandpassStokesMean[0, :])

## Plot bandpass (clean Stokes data)

In [ ]:
cleanBandpass = cleanRFI.clone()
cleanBandpass.tscrunch()
(cleanBandpassMean, cleanBandpassVariance) = cleanBandpass.get_Integration(0).baseline_stats()
cleanBandpassMaxMinVals = np.array([cleanBandpassMean[0, :].min(), cleanBandpassMean[0, :].max(), cleanBandpassMean[1, :].min(), cleanBandpassMean[1, :].max()])
cleanBandpassMean[cleanBandpassMean == 0.0] = np.nan
cleanBandpassStokes = cleanBandpass.clone()
cleanBandpassStokes.convert_state('Stokes')
cleanBandpassStokes.tscrunch()
(cleanBandpassStokesMean, cleanBandpassStokesVariance) = cleanBandpassStokes.get_Integration(0).baseline_stats()
cleanBandpassStokesMinMaxVals = np.array([cleanBandpassStokesMean[0, :].min(), cleanBandpassStokesMean[0, :].max()])
cleanBandpassStokesMean[cleanBandpassStokesMean == 0.0] = np.nan
fig, ((ax1, ax2, ax3 )) = plt.subplots(3, 1, sharex = 'col', figsize = [15, 10], tight_layout = 'false')
ax1.set_title(sourceName)
ax1.set_title('Clean data', loc = 'left')
ax1.text(np.int(nChan * 0.0125), cleanBandpassMaxMinVals.max(), 'XX', fontsize = 'large')
ax1.set_xlim(0, nChan - 1)
ax1.set_ylabel('Flux (a.u.)')
ax1.set_ylim(cleanBandpassMaxMinVals.min() - (0.1 * np.abs(cleanBandpassMaxMinVals.min())), cleanBandpassMaxMinVals.max() + (0.1 * np.abs(cleanBandpassMaxMinVals.max())))
ax1.xaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax1.plot(cleanBandpassMean[0, :])
ax2.text(np.int(nChan * 0.0125), cleanBandpassMaxMinVals.max(), 'YY', fontsize = 'large')
ax2.set_xlim(0, nChan - 1)
ax2.set_ylabel('Flux (a.u.)')
ax2.set_ylim(cleanBandpassMaxMinVals.min() - (0.1 * np.abs(cleanBandpassMaxMinVals.min())), cleanBandpassMaxMinVals.max() + (0.1 * np.abs(cleanBandpassMaxMinVals.max())))
ax2.xaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax2.plot(cleanBandpassMean[1, :])
ax3.text(np.int(nChan * 0.0125), cleanBandpassStokesMinMaxVals.max(), 'Stokes I', fontsize = 'large')
ax3.set_xlabel('Channel number')
ax3.set_ylabel('Flux (a.u.)')
ax3.set_ylim(cleanBandpassStokesMinMaxVals.min() - (0.1 * np.abs(cleanBandpassStokesMinMaxVals.min())), cleanBandpassStokesMinMaxVals.max() + (0.1 * np.abs(cleanBandpassStokesMinMaxVals.max())))
ax3.xaxis.set_ticks(np.arange(0, nChan - 1, 200))
ax3Secondary = ax3.twiny()
ax3Secondary.set_frame_on(True)
ax3Secondary.patch.set_visible(False)
ax3Secondary.xaxis.set_ticks_position('bottom')
ax3Secondary.set_xlabel('Frequency (MHz)')
ax3Secondary.xaxis.set_label_position('bottom')
ax3Secondary.spines['bottom'].set_position(('outward', 50))
ax3Secondary.set_xlim(lowFreq, highFreq)
_ = ax3Secondary.xaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 50))
ax3.plot(cleanBandpassStokesMean[0, :])

## Plot dynamic baseline spectrum (uncleaned Coherency data)

In [ ]:
dynamicSpectrum = archives[0].clone()
mean = np.zeros((nSubint, nPol, nChan))
variance = np.zeros((nSubint, nPol, nChan))
for subint in range(nSubint):
    m, v = dynamicSpectrum.get_Integration(subint).baseline_stats()
    mean[subint] = m[:, :]
meanXXMin = mean[:, 0, :].min()
meanXXMax = mean[:, 0, :].max()
meanYYMin = mean[:, 1, :].min()
meanYYMax = mean[:, 1, :].max()
meanReXYMin = mean[:, 2, :].min()
meanReXYMax = mean[:, 2, :].max()
meanImXYMin = mean[:, 3, :].min()
meanImXYMax = mean[:, 3, :].max()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = 'col', sharey = 'row', figsize = [15, 10], tight_layout = 'false')
fig.suptitle(sourceName, fontsize = 'large', y = 1.01)
fig.text(0.05, 1.0, 'Unclean data', fontsize = 'large')
ax1.set_title('XX')
ax1.set_ylabel('Channel number')
ax1.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax1.imshow(mean[:, 0, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.SymLogNorm(10, linscale = 1.0, vmin = meanXXMin, vmax = meanXXMax), interpolation = 'none')
ax2.set_title('YY')
ax2.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax2Secondary = ax2.twinx()
ax2Secondary.set_ylabel('Frequency (MHz)')
ax2Secondary.set_ylim(lowFreq, highFreq)
ax2Secondary.yaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 50))
ax2.imshow(mean[:, 1, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.SymLogNorm(10, linscale = 1.0, vmin = meanYYMin, vmax = meanYYMax), interpolation = 'none')
ax3.set_title('Re XY')
ax3.set_xlabel('Subint number')
ax3.set_ylabel('Channel number')
ax3.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax3Secondary = ax3.twiny()
ax3Secondary.set_frame_on(True)
ax3Secondary.patch.set_visible(False)
ax3Secondary.xaxis.set_ticks_position('bottom')
ax3Secondary.set_xlabel('Time (seconds)')
ax3Secondary.xaxis.set_label_position('bottom')
ax3Secondary.spines['bottom'].set_position(('outward', 50))
ax3Secondary.set_xlim(0, obsDuration)
ax3.imshow(mean[:, 2, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.SymLogNorm(10, linscale = 1.0, vmin = meanReXYMin, vmax = meanReXYMax), interpolation = 'none')
ax4.set_title('Im XY')
ax4.set_xlabel('Subint number')
ax4.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax4Secondary = ax4.twinx()
ax4Secondary.set_ylabel('Frequency (MHz)')
ax4Secondary.set_ylim(lowFreq, highFreq)
ax4Secondary.yaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 50))
ax4Tertiary = ax4.twiny()
ax4Tertiary.set_frame_on(True)
ax4Tertiary.patch.set_visible(False)
ax4Tertiary.xaxis.set_ticks_position('bottom')
ax4Tertiary.set_xlabel('Time (seconds)')
ax4Tertiary.xaxis.set_label_position('bottom')
ax4Tertiary.spines['bottom'].set_position(('outward', 50))
ax4Tertiary.set_xlim(0, obsDuration)
ax4.imshow(mean[:, 3, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.SymLogNorm(10, linscale = 1.0, vmin = meanImXYMin, vmax = meanImXYMax), interpolation = 'none')

## Plot dynamic baseline spectrum (cleaned Stokes data)

In [ ]:
dynamicSpectrum = cleanRFI.clone()
dynamicSpectrum.convert_state('Stokes')
mean = np.zeros((nSubint, nPol, nChan))
variance = np.zeros((nSubint, nPol, nChan))
for subint in range(nSubint):
    m, v = dynamicSpectrum.get_Integration(subint).baseline_stats()
    variance[subint] = v[:, :]
varianceXXMin = variance[:, 0, :].min()
varianceXXMax = variance[:, 0, :].max()
varianceYYMin = variance[:, 1, :].min()
varianceYYMax = variance[:, 1, :].max()
varianceReXYMin = variance[:, 2, :].min()
varianceReXYMax = variance[:, 2, :].max()
varianceImXYMin = variance[:, 3, :].min()
varianceImXYMax = variance[:, 3, :].max()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = 'col', sharey = 'row', figsize = [15, 10], tight_layout = 'false')
fig.suptitle(sourceName, fontsize = 'large', y = 1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize = 'large')
ax1.set_title('I')
ax1.set_ylabel('Channel number')
ax1.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax1.imshow(variance[:, 0, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.Normalize(vmin = varianceXXMin, vmax = varianceXXMax), interpolation = 'none')
ax2.set_title('Q')
ax2.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax2Secondary = ax2.twinx()
ax2Secondary.set_ylabel('Frequency (MHz)')
ax2Secondary.set_ylim(lowFreq, highFreq)
ax2Secondary.yaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 50))
ax2.imshow(variance[:, 1, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.Normalize(vmin = varianceYYMin, vmax = varianceYYMax), interpolation = 'none')
ax3.set_title('U')
ax3.set_xlabel('Subint number')
ax3.set_ylabel('Channel number')
ax3.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax3Secondary = ax3.twiny()
ax3Secondary.set_frame_on(True)
ax3Secondary.patch.set_visible(False)
ax3Secondary.xaxis.set_ticks_position('bottom')
ax3Secondary.set_xlabel('Time (seconds)')
ax3Secondary.xaxis.set_label_position('bottom')
ax3Secondary.spines['bottom'].set_position(('outward', 50))
ax3Secondary.set_xlim(0, obsDuration)
ax3.imshow(variance[:, 2, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.Normalize(vmin = varianceReXYMin, vmax = varianceReXYMax), interpolation = 'none')
ax4.set_title('V')
ax4.set_xlabel('Subint number')
ax4.yaxis.set_ticks(np.arange(0, nChan - 1, 400))
ax4Secondary = ax4.twinx()
ax4Secondary.set_ylabel('Frequency (MHz)')
ax4Secondary.set_ylim(lowFreq, highFreq)
ax4Secondary.yaxis.set_ticks(np.arange(np.rint(lowFreq), np.rint(highFreq), 50))
ax4Tertiary = ax4.twiny()
ax4Tertiary.set_frame_on(True)
ax4Tertiary.patch.set_visible(False)
ax4Tertiary.xaxis.set_ticks_position('bottom')
ax4Tertiary.set_xlabel('Time (seconds)')
ax4Tertiary.xaxis.set_label_position('bottom')
ax4Tertiary.spines['bottom'].set_position(('outward', 50))
ax4Tertiary.set_xlim(0, obsDuration)
ax4.imshow(variance[:, 3, :].T, origin = 'lower', aspect = 'auto', cmap = cm.afmhot, norm = colors.Normalize(vmin = varianceImXYMin, vmax = varianceImXYMax), interpolation = 'none')

## Plot signal-to-noise ratio vs. subintegration (cleaned Coherency data)

In [ ]:
archiveSNR = commonCleanRFI.clone()
newIntegration = archiveSNR.get_Integration(0).total()
snrData = np.zeros(nSubint)
for iSubintegration in range (1, nSubint):
    nextIntegration = archiveSNR.get_Integration(iSubintegration).total()
    newIntegration.combine(nextIntegration)
    profile = newIntegration.get_Profile(0, 0)
    snrData[iSubintegration] = profile.snr()
fig, ax1 = plt.subplots(1, 1, sharex = 'col', sharey = 'row', figsize = [15, 5], tight_layout = 'false')
fig.suptitle(sourceName, fontsize = 'large', y = 1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize = 'large')
ax1.set_xlabel('Subint number')
ax1.set_ylabel('Signal-to-noise ratio')
ax1.set_xlim(0, nSubint)
ax1Secondary = ax1.twiny()
ax1Secondary.set_frame_on(True)
ax1Secondary.patch.set_visible(False)
ax1Secondary.xaxis.set_ticks_position('bottom')
ax1Secondary.set_xlabel('Time (seconds)')
ax1Secondary.xaxis.set_label_position('bottom')
ax1Secondary.spines['bottom'].set_position(('outward', 50))
ax1Secondary.set_xlim(0, obsDuration)
ax1.plot(snrData)

In [ ]:
print 'Report Ended:'
!date